# Animation Format

by Jerome Eippers, 2024

## Skeleton structure 
A character is composed of a hiearchy of joint. Each with a local transfrom relative to its parent.
       
```
Root
└── Hips
    ├── Spine
    │   └── Chest
    │       ├── Neck
    │       │   └── Head
    │       ├── LeftShoulder
    │       │   └── LeftArm
    │       │       └── (...)
    │       └── RightShoulder
    │           └── RightArm
    │               └── (...)
    ├── LeftUpLeg
    │   └── LeftLeg
    │       └── LeftFoot
    └── RightUpLeg
        └── RightLeg
            └── RightFoot
```

## Animation representation

$$
M(t) = (\textbf{p}_0(t), \textbf{q}_0(t), \textbf{q}_1(t), \textbf{q}_2(t), \dots, \textbf{q}_b(t))
$$
  
An articulated objects such as human figures are usually represented
as rotation hierarchies parameterized by a whole-body translation $\textbf{p}_0$, a
whole-body rotation $\textbf{q}_0$, and a set of joint angles $\textbf{q}_1, \dots, \textbf{q}_b$.   
A motion $M$ is described by a set of motion curves each giving the value of one of the model’s parameters as a function of time.

### Framework

In our case, we use the animation format from Ubisoft Lafan1 with a motion being parametrized by the position and orientation of all joints.

$$
\begin{align*}
M(t) &= (\textbf{P}(t), \textbf{Q}(t))\\
\textbf{P}(t) &= (\textbf{p}_0(t), \dots, \textbf{p}_b(t))\\
\textbf{Q}(t) &= (\textbf{q}_0(t), \dots, \textbf{q}_b(t))\\
\end{align*}
$$
With $t$ beeing discreet at 30 frame per second.  

This gives us 2 tensors of shapes :

* $\textbf{P}$ = np.array[frame count, bone count, 3]
* $\textbf{Q}$ = np.array[frame count, bone count, 4]


In [1]:
import numpy as np
from ipywidgets import widgets, interact

import ipyanimlab as lab

viewer = lab.Viewer(move_speed=5, width=1280, height=720)

# Load Character

Mesh and Skeleton.

In [2]:
character = viewer.import_usd_asset('AnimLabSimpleMale.usd')

In [4]:
viewer.begin_shadow()
viewer.draw(character)
viewer.end_shadow()

viewer.begin_display()
viewer.draw_ground()
viewer.draw(character)
viewer.end_display()

viewer.disable(depth_test=True)
viewer.draw_axis(character.world_skeleton_xforms(), 5)
viewer.draw_lines(character.world_skeleton_lines())
viewer.execute_commands()
viewer

Viewer(camera_far=2800.0, camera_near=20.0, camera_pitch=-14.400000000000006, camera_pos=[-18.6193078352162, 1…

# Load Animation

## load the data

In [9]:
animation = lab.import_bvh('../../resources/lafan1/bvh/aiming1_subject1.bvh')

In [10]:
print('positions :', animation.pos.shape)
print('quaternions :', animation.quats.shape)

positions : (7184, 22, 3)
quaternions : (7184, 22, 4)


In [12]:
def render(frame):
    
    p = (animation.pos[frame,...]).copy()
    q = (animation.quats[frame,...]).copy()

    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(a[1,:3,3])
    
    viewer.begin_shadow()
    viewer.draw(character, a, animation.bones)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a, animation.bones)
    viewer.end_display()
    
    viewer.disable(depth_test=True)
        
    viewer.draw_axis(character.world_skeleton_xforms(a, animation.bones), 5)
    viewer.draw_lines(character.world_skeleton_lines(a, animation.bones))
    
    viewer.execute_commands()
    
interact(render, frame=lab.Timeline(max=animation.quats.shape[0]-1))
viewer

interactive(children=(Timeline(value=0, children=(Play(value=0, description='play', interval=33, layout=Layout…

Viewer(camera_far=2800.0, camera_near=20.0, camera_pitch=-10.600000000000007, camera_pos=[-151.6991414754351, …

## Match the skeleton translations

In [13]:
animmap = lab.AnimMapper(character)
animation = lab.import_bvh('../../resources/lafan1/bvh/aiming1_subject1.bvh', anim_mapper=animmap)

In [17]:
def render(frame):
    
    p = (animation.pos[frame,...]).copy()
    q = (animation.quats[frame,...]).copy()

    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(a[1,:3,3])
    
    viewer.begin_shadow()
    viewer.draw(character, a, animation.bones)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a, animation.bones)
    viewer.end_display()
    
    viewer.disable(depth_test=True)
        
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))
    
    viewer.execute_commands()
    
interact(render, frame=lab.Timeline(max=animation.quats.shape[0]-1))
viewer

interactive(children=(Timeline(value=0, children=(Play(value=0, description='play', interval=33, layout=Layout…

Viewer(camera_far=2800.0, camera_near=20.0, camera_pitch=-13.800000000000011, camera_pos=[-151.6991414754351, …

## Project Root

We will project the root under the Hips.

In [19]:
frame_count = animation.quats.shape[0]
bone_count = animation.quats.shape[1]

hips_v = lab.utils.quat_mul_vec(animation.quats[:, 1], np.asarray([0,1,0], dtype=np.float32)[np.newaxis,...].repeat(frame_count, axis=0))
angle = np.arctan2(hips_v[:, 0], hips_v[:, 2])/2
root_q = np.zeros([frame_count, 4], dtype=np.float32)
root_q[:, 0] = np.cos(angle)
root_q[:, 2] = np.sin(angle)
root_p = np.zeros([frame_count, 3], dtype=np.float32)
root_p[:, [0,2]] = animation.pos[:, 1, [0,2]]

animation.quats[:, 1], animation.pos[:, 1] = lab.utils.qp_mul( lab.utils.qp_inv((root_q, root_p)), (animation.quats[:, 1], animation.pos[:, 1]) )
animation.quats[:, 0], animation.pos[:, 0] = root_q, root_p

In [21]:
def render(frame, static_position=False, static_rotation=False):
    
    p = (animation.pos[frame,...]).copy()
    q = (animation.quats[frame,...]).copy()
    
    if static_position:
        p[0] = np.zeros(3, dtype=np.float32)
    if static_rotation:
        q[0] = np.asarray([1,0,0,0], dtype=np.float32)
    
    a = lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(a[1,:3,3])
    
    viewer.begin_shadow()
    viewer.draw(character, a, animation.bones)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a, animation.bones)
    viewer.end_display()
    
    viewer.disable(depth_test=True)
        
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))
    
    viewer.execute_commands()
    
interact(render, 
         static_position=widgets.Checkbox(), 
         static_rotation=widgets.Checkbox(), 
         frame=lab.Timeline(max=animation.quats.shape[0]-1))
viewer

interactive(children=(Timeline(value=0, children=(Play(value=0, description='play', interval=33, layout=Layout…

Viewer(camera_far=2800.0, camera_near=20.0, camera_pitch=-13.800000000000011, camera_pos=[-151.6991414754351, …